In [27]:
import pandas as pd
import numpy as np
import folium
from folium.plugins import MarkerCluster
import random
from datetime import datetime
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display, clear_output


In [28]:
# Define truck types with their capabilities and possible equipment
truck_types = [
    {
        'Truck_ID': 'T1',
        'Truck_Type': 'Multilift 4 Axles',
        'Category': ['Waste', 'Enrobée'],
        'Capabilities': ['Standard Access'],
        'Possible_Equipment': ['Benne 10 m³', 'Benne 20 m³', 'Thermos', 'Pont 13 m³'],
        'Available': True
    },
    {
        'Truck_ID': 'T2',
        'Truck_Type': 'Multilift 4 Axles',
        'Category': ['Waste', 'Enrobée'],
        'Capabilities': ['Standard Access'],
        'Possible_Equipment': ['Benne 10 m³', 'Benne 20 m³', 'Thermos', 'Pont 13 m³'],
        'Available': True
    },
    {
        'Truck_ID': 'T3',
        'Truck_Type': 'Multilift 4 Axles Tridem',
        'Category': ['Waste', 'Enrobée'],
        'Capabilities': ['Difficult Access'],
        'Possible_Equipment': ['Benne 10 m³', 'Benne 20 m³', 'Thermos', 'Pont 13 m³'],
        'Available': True
    },
    {
        'Truck_ID': 'T4',
        'Truck_Type': 'Camion-grue 4 Axles',
        'Category': ['Grue'],
        'Capabilities': ['Equipped with Crane'],
        'Possible_Equipment': ['Grue', 'Remorque Tandem'],
        'Available': True
    },
    {
        'Truck_ID': 'T5',
        'Truck_Type': 'Multilift 4 Axles',
        'Category': ['Waste'],
        'Capabilities': ['Standard Access'],
        'Possible_Equipment': ['Pont 13 m³'],
        'Available': True
    }
]

# Convert to DataFrame for better handling
trucks_df = pd.DataFrame(truck_types)
trucks_df


,Truck_ID,Truck_Type,Category,Capabilities,Possible_Equipment,Available
0,T1,Multilift 4 Axles,"[Waste, Enrobée]",[Standard Access],"[Benne 10 m³, Benne 20 m³, Thermos, Pont 13 m³]",True
1,T2,Multilift 4 Axles,"[Waste, Enrobée]",[Standard Access],"[Benne 10 m³, Benne 20 m³, Thermos, Pont 13 m³]",True
2,T3,Multilift 4 Axles Tridem,"[Waste, Enrobée]",[Difficult Access],"[Benne 10 m³, Benne 20 m³, Thermos, Pont 13 m³]",True
3,T4,Camion-grue 4 Axles,[Grue],[Equipped with Crane],"[Grue, Remorque Tandem]",True
4,T5,Multilift 4 Axles,[Waste],[Standard Access],[Pont 13 m³],True


In [29]:
# Define client needs without specifying equipment
client_needs = ['Waste', 'Enrobée', 'Grue']

# Define time windows
time_windows = [
    ('Morning', '08:00', '12:00'),
    ('Afternoon', '13:00', '17:00'),
    ('All Day', '08:00', '17:00')
]


In [30]:
# Paris geographical boundaries
paris_bounds = {
    'lat_min': 48.815573,
    'lat_max': 48.902144,
    'lon_min': 2.224199,
    'lon_max': 2.469920
}


In [31]:
def generate_random_location(num_locations):
    latitudes = np.random.uniform(paris_bounds['lat_min'], paris_bounds['lat_max'], num_locations)
    longitudes = np.random.uniform(paris_bounds['lon_min'], paris_bounds['lon_max'], num_locations)
    return pd.DataFrame({
        'Latitude': latitudes,
        'Longitude': longitudes
    })


In [32]:
def generate_clients(num_clients, waste_ratio=0.6, enrobee_ratio=0.3, grue_ratio=0.1,
                     difficult_access_ratio=0.2, time_window_distribution=None):
    """
    Generates fake client data without assigning required equipment.
    """
    if time_window_distribution is None:
        time_window_distribution = {'Morning': 0.4, 'Afternoon': 0.4, 'All Day': 0.2}
    
    # Ensure the ratios sum up to 1
    total_ratio = waste_ratio + enrobee_ratio + grue_ratio
    if total_ratio != 1.0:
        print("Adjusting ratios to sum up to 1.0")
        waste_ratio /= total_ratio
        enrobee_ratio /= total_ratio
        grue_ratio /= total_ratio

    # Assign client needs based on ratios
    needs = np.random.choice(client_needs,
                             size=num_clients,
                             p=[waste_ratio, enrobee_ratio, grue_ratio])

    # Assign access difficulty
    access_difficulty = np.random.choice(['Yes', 'No'],
                                         size=num_clients,
                                         p=[difficult_access_ratio, 1 - difficult_access_ratio])

    # Assign time windows based on distribution
    time_window_labels = list(time_window_distribution.keys())
    time_window_probs = list(time_window_distribution.values())
    time_window = np.random.choice(time_window_labels,
                                   size=num_clients,
                                   p=time_window_probs)

    # Generate random locations
    locations = generate_random_location(num_clients)

    # Assign volumes based on client needs
    volumes_t = []
    volumes_kg = []
    for need in needs:
        if need == 'Waste':
            volume = np.random.uniform(10, 20)  # in tonnes
        elif need == 'Enrobée':
            volume = np.random.uniform(13, 18)  # in tonnes
        elif need == 'Grue':
            volume = np.random.uniform(8.5, 17)  # in tonnes
        volumes_t.append(round(volume, 2))
        volumes_kg.append(round(volume * 1000, 2))  # Convert to kg

    clients_df = pd.DataFrame({
        'Client_ID': [f'C{str(i).zfill(3)}' for i in range(1, num_clients + 1)],
        'Need': needs,
        'Volume_t': volumes_t,
        'Volume_kg': volumes_kg,
        'Access_Difficulty': access_difficulty,
        'Time_Window_Label': time_window,
        'Latitude': locations['Latitude'],
        'Longitude': locations['Longitude']
    })

    return clients_df


In [33]:
def map_time_windows(clients_df):
    time_window_map = {
        'Morning': ('08:00', '12:00'),
        'Afternoon': ('13:00', '17:00'),
        'All Day': ('08:00', '17:00')
    }
    
    clients_df['Time_Window_Start'] = clients_df['Time_Window_Label'].map(lambda x: time_window_map[x][0])
    clients_df['Time_Window_End'] = clients_df['Time_Window_Label'].map(lambda x: time_window_map[x][1])
    
    return clients_df


In [34]:
# Define widgets
num_clients_widget = widgets.IntSlider(value=50, min=10, max=200, step=10,
                                       description='Number of Clients:', continuous_update=False)

waste_ratio_widget = widgets.FloatSlider(value=0.6, min=0.0, max=1.0, step=0.05,
                                         description='Waste Ratio:', continuous_update=False)

enrobee_ratio_widget = widgets.FloatSlider(value=0.3, min=0.0, max=1.0, step=0.05,
                                           description='Enrobée Ratio:', continuous_update=False)

grue_ratio_widget = widgets.FloatSlider(value=0.1, min=0.0, max=1.0, step=0.05,
                                        description='Grue Ratio:', continuous_update=False)

difficult_access_ratio_widget = widgets.FloatSlider(value=0.2, min=0.0, max=1.0, step=0.05,
                                                   description='Difficult Access %:', continuous_update=False)

@interact(num_clients=num_clients_widget,
          waste_ratio=waste_ratio_widget,
          enrobee_ratio=enrobee_ratio_widget,
          grue_ratio=grue_ratio_widget,
          difficult_access_ratio=difficult_access_ratio_widget)
def generate_data(num_clients, waste_ratio, enrobee_ratio, grue_ratio, difficult_access_ratio):
    # Validate that ratios sum to 1.0
    total_ratio = waste_ratio + enrobee_ratio + grue_ratio
    if total_ratio != 1.0:
        print("Adjusting ratios to sum up to 1.0")
        waste_ratio /= total_ratio
        enrobee_ratio /= total_ratio
        grue_ratio /= total_ratio
    
    # Generate clients
    clients = generate_clients(
        num_clients=num_clients,
        waste_ratio=waste_ratio,
        enrobee_ratio=enrobee_ratio,
        grue_ratio=grue_ratio,
        difficult_access_ratio=difficult_access_ratio
    )
    
    # Map time windows
    clients = map_time_windows(clients)
    
    # Display the first few rows of the generated data
    display(clients.head())
    
    # Store the generated clients in a global variable for later use
    global generated_clients
    generated_clients = clients


interactive(children=(IntSlider(value=50, continuous_update=False, description='Number of Clients:', max=200, …

In [35]:
def visualize_clients(clients_df):
    # Initialize Folium map centered around Paris
    paris_map = folium.Map(location=[48.8566, 2.3522], zoom_start=12)
    
    # Define color mapping based on client needs
    color_map = {
        'Waste': 'blue',
        'Enrobée': 'green',
        'Grue': 'red'
    }
    
    marker_cluster = MarkerCluster().add_to(paris_map)
    
    for idx, row in clients_df.iterrows():
        popup_text = (f"Client ID: {row['Client_ID']}<br>"
                      f"Need: {row['Need']}<br>"
                      f"Volume: {row['Volume_kg']} kg<br>"
                      f"Access Difficulty: {row['Access_Difficulty']}<br>"
                      f"Time Window: {row['Time_Window_Start']} - {row['Time_Window_End']}")
        
        icon = folium.Icon(color=color_map.get(row['Need'], 'gray'), icon='info-sign')
        
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=popup_text,
            icon=icon
        ).add_to(marker_cluster)
    
    display(paris_map)


In [36]:
visualize_button = widgets.Button(description="Visualize Clients on Map", button_style='success')

output_map = widgets.Output()

def on_button_click(b):
    with output_map:
        clear_output()
        if 'generated_clients' in globals():
            visualize_clients(generated_clients)
        else:
            print("Please generate the data first.")

visualize_button.on_click(on_button_click)

display(visualize_button, output_map)


Button(button_style='success', description='Visualize Clients on Map', style=ButtonStyle())

Output()

In [37]:
def display_summary(clients_df):
    total = len(clients_df)
    needs_counts = clients_df['Need'].value_counts()
    access_counts = clients_df['Access_Difficulty'].value_counts()
    time_window_counts = clients_df['Time_Window_Label'].value_counts()
    
    print(f"Total Clients: {total}\n")
    print("Clients per Need:")
    display(needs_counts)
    print("\nAccess Difficulty:")
    display(access_counts)
    print("\nTime Windows:")
    display(time_window_counts)


In [38]:
summary_button = widgets.Button(description="Display Summary", button_style='info')

output_summary = widgets.Output()

def on_summary_click(b):
    with output_summary:
        clear_output()
        if 'generated_clients' in globals():
            display_summary(generated_clients)
        else:
            print("Please generate the data first.")

summary_button.on_click(on_summary_click)

display(summary_button, output_summary)


Button(button_style='info', description='Display Summary', style=ButtonStyle())

Output()

In [39]:
export_button = widgets.Button(description="Export to JSON", button_style='warning')

output_export = widgets.Output()

def on_export_click(b):
    with output_export:
        clear_output()
        if 'generated_clients' in globals():
            # Reorder columns for clarity
            export_df = generated_clients.copy()
            
            export_df = export_df[[
                'Client_ID',
                'Need',
                'Volume_t',
                'Volume_kg',
                'Access_Difficulty',
                'Time_Window_Label',
                'Time_Window_Start',
                'Time_Window_End',
                'Latitude',
                'Longitude'
            ]]
            
            # Export to JSON with proper encoding
            filename = f"clients_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
            export_df.to_json(filename, orient='records', lines=False, indent=4, force_ascii=False)
            print(f"Data exported to {filename}")
        else:
            print("Please generate the data first.")

export_button.on_click(on_export_click)

display(export_button, output_export)


Button(button_style='warning', description='Export to JSON', style=ButtonStyle())

Output()